# 1. All Paths from source to target (DAG)

a directed edge from node i to node graph[i][j] 

In [20]:
# 0부터 모든 노드까지 가능한 Paths 반환하기
def allPathsSourceTarget(graph):
    
    # graph 정보 있는 Dictionary 만들기
    G = dict()
    for i in range(len(graph)):
            G[i] = graph[i]
    # print(G)
    
    # 각 노드와 해당 노드까지의 경로 저장
    stack = [(0, [0])]
    allpaths = []
    # Acyclic path 찾아야 하므로 DFS 이용
    while stack:
        cur, path = stack.pop()
        
        # 더이상 갈 노드가 없다면 (목적지라면)
        if len(G[cur]) == 0:
            # print('stack: ', stack)
            allpaths.append(path)
            continue
    
        for adj in G[cur]:
            if adj not in path:
                stack.append((adj, path+[adj]))
    return allpaths
    
    

In [18]:
graph = [[1,2],[3],[3],[]]
allPathsSourceTarget(graph)

{0: [1, 2], 1: [3], 2: [3], 3: []}
stack:  [(1, [0, 1])]
stack:  []


[[0, 2, 3], [0, 1, 3]]

In [21]:
graph = [[4,3,1],[3,2,4],[3],[4],[]]
allPathsSourceTarget(graph)

{0: [4, 3, 1], 1: [3, 2, 4], 2: [3], 3: [4], 4: []}
stack:  [(4, [0, 4]), (3, [0, 3]), (3, [0, 1, 3]), (2, [0, 1, 2])]
stack:  [(4, [0, 4]), (3, [0, 3]), (3, [0, 1, 3])]
stack:  [(4, [0, 4]), (3, [0, 3])]
stack:  [(4, [0, 4])]
stack:  []


[[0, 1, 4], [0, 1, 2, 3, 4], [0, 1, 3, 4], [0, 3, 4], [0, 4]]

# 1334. Find the City With the Smallest Number of Neighbors at a Threshold

<img src = "/Users/soungmunkim/Desktop/Coding/QualExamCoding/soungmunkim/Python/Graph/z_img/z_문제스샷.png">

In [7]:
import sys
import numpy as np

# 연결되지 않은 path의 weight
INF = sys.maxsize

 #--------------------- Floyd warhshall 알고리즘 구현 (모든 source node에 대해 shortest path -----------------------
# Floyd-Warshall Algorithm
# input - adj(인접행렬), n(vertex의 수)
# output - dist : 최소 i->j의 path를 기록한 matrix (shortest path distance 기록)
# output - pred : i->j의 path에서 j 직전에 거치는 node를 기록한 matrix. (shortest path node기록)
def Floyd_warshall(adj,n):
    # 거리 기록할 배열 초기화
    dist = adj

    # predecessor 기록할 배열 초기화
    pred = [['NIL']*n for i in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j and adj[i][j] != INF:
                pred[i][j] = i

    # intermediate node 'k'를 하나씩 추가하며, 해당 경우의 최단 거리 계산
    for k in range(n):
        for i in range(n):   # i: 출발노드
            for j in range(n):   # j: 도착노드

                # k를 추가함으로써 이전보다 더 빠른 길이 생기는 경우의 처리
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    pred[i][j] = pred[k][j]
    return dist, pred

# shortest path list에 담아 출력하기
def print_shortest_path(pred, i, j):
    path = []
    def recursion(pred, i, j):
        if i == j:
            # print(i, end=' ')
            path.append(i)
            return
        recursion(pred, i, pred[i][j])
        # print(j, end=' ')
        path.append(j)   
    
    recursion(pred, i, j)
    return path
    

def findTheCity(n, edges, distanceThreshold):
    # Row, Col = len(edges), len(edges)
    adj = np.zeros((n, n))
    
    #--------------------- bidirectical adj.matrix 만들기-----------------------
    for i in range(n): # 한 방향
        for j in range(n):
            # 먼저 같은 노드가 아니면 INf.로 initialize 하기
            if i != j:
                adj[i][j] = float('inf')
            
            from_node = edges[i][0]
            to_node = edges[i][1]
            dist = edges[i][2]
            adj[from_node][to_node] = dist
            
    for i in range(n): # 다른 한 방향
        for j in range(n):
            if i != j:
    
                from_node = edges[i][1]
                to_node = edges[i][0]
                dist = edges[i][2]
                adj[from_node][to_node] = dist
    #----------------------------------------------------------------------------             
            
    # print("adj. matrix: \n", adj) # 완성된 bidrectical adj.matrix
    
    distance, pred = Floyd_warshall(adj, n)  # 여기서 Path는 프린트 할 필요 없음 (pred 사용 안 함)
    # print("distance matrix: \n", distance)  
    print("pred matrix: \n", pred)
    
    shortestpaths = dict()
    allpaths = []
    for i in range(len(distance)):
        for j in range(len(distance[0])):
            # i -> j까지의 최단 거리이므로
        
            if i != j and distance[i][j] <= distanceThreshold:
                # shortest path에 각 Source마다 shortest distance가 4이하인 도착점 넣기
                print((i,j), end='\t')
                if i not in shortestpaths:
                    shortestpaths[i] = [j]
                else:
                    shortestpaths[i].append(j)
                print(print_shortest_path(pred, i, j))
                # allpaths.append(print_shortest_path(pred, i, j))
                
                    
    # print("all paths: ", allpaths)
    
    min_num = 100
    max_city = -1
    # print("shortestpath: \n", shortestpaths)
    
    # 가장 적은 number of cities중 가장 큰 Source node return 하기
    for city, adj in shortestpaths.items():
        if len(adj) <= min_num:
            min_num = len(adj)
            if city > max_city:
                max_city = city
    
    return max_city     
    

In [8]:
n = 4
edges = [[0,1,3],[1,2,1],[1,3,4],[2,3,1]]
distanceThreshold = 4
findTheCity(n, edges, distanceThreshold)

pred matrix: 
 [['NIL', 0, 1, 2], [1, 'NIL', 1, 2], [1, 2, 'NIL', 2], [1, 2, 3, 'NIL']]
(0, 1)	[0, 1]
(0, 2)	[0, 1, 2]
(1, 0)	[1, 0]
(1, 2)	[1, 2]
(1, 3)	[1, 2, 3]
(2, 0)	[2, 1, 0]
(2, 1)	[2, 1]
(2, 3)	[2, 3]
(3, 1)	[3, 2, 1]
(3, 2)	[3, 2]


3